In [17]:
import pandas as pd
import numpy as np
import requests

from pygeocoder import Geocoder
import geopandas as gpd
import pgeocode
from shapely.geometry import Point, Polygon

from uszipcode import SearchEngine

import matplotlib
import matplotlib.pyplot as plt

In [18]:
#import the data
data = pd.read_csv('flaviar^Mcaskers_orders_USD_2016-2020_PY.csv',parse_dates=True,index_col='customer_id')

In [19]:
#Clean up the data
#Set dates to correct format
data['customer_creation_date']=pd.to_datetime(data['customer_creation_date'], format="%m/%d/%Y")
data['order_transaction_date']=pd.to_datetime(data['order_transaction_date'], format="%m/%d/%Y")

#Drop unneeded columns
data = data.drop(columns=['currency','order_id','transaction_type','customer_creation_date','total_qty','total'])

#Drop bad rows (where zip code is messed up or contains non numeric vals)
data=data[data['zip'].astype(str).str.isdigit()]
#data.zip=data.zip.astype(int)
data = data[data['zip'].apply(lambda x: len(str(x)) > 4)]
data = data[data['zip'].apply(lambda x: len(str(x)) < 8)]

In [6]:
#create a pivot table that counts the occurence of each zip code
data=data.pivot_table(index='zip',aggfunc='count')
data=data.rename(columns={"order_transaction_date":"Total"})

In [7]:
#TEMPORAR: FIX THIS LATER!
data['zips']=data.index

In [ ]:
#split data into years - 
#THIS WILL BE IMPLEMENTED LATER. DOES NOT CURRENTLY WORK
coln='customer_creation_date'
data19 = data[(data[coln] > '12/31/2018')&(data[coln]<='12/31/2019')]
data18 = data[(data[coln] > '12/31/2017')&(data[coln]<='12/31/2018')]

In [8]:
search=SearchEngine(simple_zipcode=True)

In [20]:
data.dtypes

order_transaction_date    datetime64[ns]
zip                               object
dtype: object

In [9]:
sample=data
states=[]
for index,row in sample.iterrows():
    states.append(search.by_zipcode(row['zips']).state)
sample['state']=states

In [14]:
data

,Total,zips,state
zip,,,
10000,1,10000,None
100000,1,100000,None
10001,388,10001,NY
10002,181,10002,NY
10003,315,10003,NY
10004,128,10004,NY
10005,160,10005,NY
10006,104,10006,NY
10007,142,10007,NY


In [ ]:
data=sample.pivot_table(index='state',values='Total',aggfunc=np.sum)

Get the Map

In [ ]:
#load a basemap of the US. map will be stored as country
shapefile = "gz_2010_us_040_00_5m.json"
country=gpd.read_file(shapefile)

In [ ]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
country=country.replace({"NAME": us_state_abbrev})

In [ ]:
data['NAME']=data.index

Merge data & plot

In [ ]:
merged=country.set_index('NAME').join(data.set_index('NAME'))

In [ ]:
variable = 'Total'
vmin,vmax=120,220

In [ ]:
merged = merged.drop(['AL','AK','AL', 'AR', 'DE', 'HI', 'IA', 'ID', 'IL', 'KY', 'MD', 'MI', 'MN', 'MS', 'MT', 'ND', 'OH', 'OK', 'PA', 'PR', 'SD', 'TX', 'UT', 'VA', 'VT', 'WV', 'WY'])

In [ ]:
x=merged
x['states']=x.index

In [ ]:
x['normalized']=np.log(x['Total'])

In [ ]:
x

In [ ]:
import plotly.express as px

fig = px.choropleth(x, geojson=states, locations='statess', 
                           color='normalized',
                           locationmode='USA-states',
                           color_continuous_scale="Viridis",
                           range_color=(8, 12),
                           scope="usa",
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
x['normalized']=np.log(x['Total'])

In [ ]:
x.min(axis=0)

In [ ]:
x.max(axis=0)

In [ ]:
fig = px.choropleth(x, geojson=states, locations='statess', 
                           color='normalized',
                           locationmode='USA-states',
                           color_continuous_scale="Viridis",
                           range_color=(1, 15),
                           scope="usa",
                          )
fig.update_layout(
    title_text='Flaviar Order Breakdown',
    margin={"r":0,"l":0,"b":0})
fig.show()

In [ ]:
#OUTDATED
#RETRIEVE LONGITUDES AND LATITUDES FOR EACH ZIP CODE

#THIS METHOD FINDS THE EXACT COORDINATES. WE WILL ACTUALLY NEED TO
#USE THE STATES. THIS BOX IS CURRENLTY ON HOLD

sample=data.sample(1000)
longs=[]
lats=[]
for index,row in sample.iterrows():
    longs.append(search.by_zipcode(row['zips']).lng)
    lats.append(search.by_zipcode(row['zips']).lat)
sample['long']=longs
sample['lat']=lats

In [ ]:
merged=country.set_index('NAME').join(findata.set_index('NAME'))

In [ ]:
#OUTDATED
#THIS IS BASED ON COORDINATES TOO. WE WILL NOW BE USING STATES (SEE CELL ABOVE)
sampl=sample

indexNames = sampl[ sample['long'] < -135 ].index
sampl.drop(indexNames,inplace=True,)

sampl['coordinates']=sampl[['long','lat']].values.tolist()
sampl['coordinates']=sampl['coordinates'].apply(Point)
sampl = gpd.GeoDataFrame(sampl, geometry='coordinates')

In [ ]:
base

In [ ]:
base=country[country['NAME'].isin(['Alaska','Hawaii']) == False].plot(figsize=(30,20), color='#3B3C6E');



In [ ]:
sampl.rename(columns={'coordinates':'geometry'},inplace=True)

In [ ]:
type(base.geometry[0])

In [ ]:
points_clean = sampl[points.geometry.type == 'Point']

In [ ]:
base=country[country['NAME'].isin(['Alaska','Hawaii']) == False]
merged=gpd.sjoin(sampl,base,op='within')

#base.set_index('geometry').join(sampl.set_index('coordinates'))
#base.iloc[0].geometry

In [ ]:
#variable = 'Total'
#vmin,vmax = 120,220

base=country[country['NAME'].isin(['Alaska','Hawaii']) == False].plot(figsize=(30,20), color='#3B3C6E');

sampl.plot(ax=base, color='red',cmap='Blues',linewidth=0.8, marker="*", markersize=20);